In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT-3.5 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def generate_response(paragraph, query):
    # Concatenate the paragraph and query
    input_text = f"Paragraph: {paragraph}\nQuery: {query}"
    
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    # Generate the response
    output = model.generate(input_ids, max_length=200, num_return_sequences=1, do_sample=True, top_k=50, top_p=0.95, num_beams=1)
    
    # Decode the output
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response

# Example usage
paragraph = "A person eating a yellow banana in a spaceship. This man looks happy and satasfying and am trying to sit on the chair, but due lack of gravity, he could not."
query = "Base on the paragraph, What is the man doing and where is he at?"
response = generate_response(paragraph, query)
print(response)

C:\Users\\MiniConda3\envs\VLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Paragraph: A person eating a yellow banana in a spaceship. This man looks happy and satasfying and am trying to sit on the chair, but due lack of gravity, he could not.
Query: Base on the paragraph, What is the man doing and where is he at?
Replyer: The pilot is looking at an area where the cabin and the other rooms of the spaceship are connected. The ship is in the middle of the highway, and no pilot and the space station are visible. The pilot is seated at the seat of one of the spaceships, and the space station is near to the left and center of the space station. There are two astronauts waiting for the space station in front of the left and right spaceships.
Question: Is this man sleeping in the other spaceships, or in another of the ships?
Answer: This man is waiting at an altitude of 100 kilometers and is seated in the center of a space station. He has a


In [ ]:
import requests

def generate_response(paragraph, query):
    # Concatenate the paragraph and query
    input_text = f"Paragraph: {paragraph}\nQuery: {query}"

    # Set up the request parameters
    data = {
        "prompt": input_text,
        "max_tokens": 200,
        "n": 1,
        "stop": None,
        "temperature": 0.7,
        "model": "gpt-3.5-turbo-0125"
    }

    # Set up the OpenAI API endpoint and headers
    endpoint = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer sk-proj-"
    }

    # Make the request to the API
    response = requests.post(endpoint, json=data, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the generated text
        generated_text = response.json()["choices"][0]["text"].strip()
        return generated_text
    else:
        # Handle any errors
        print(f"Error: {response.text}")
        return None

# Example usage
paragraph = "Extract the intent and arguments by performing semantic parsing of the following sentence. Give the output in JSON format:"
query = "Base on the paragraph, What is the man doing and where is he at?"
response = generate_response(paragraph, query)
print(response)


Error: {
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}

None


In [2]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
torch.cuda.empty_cache()
# Specify the model path
model_path = 'mtgv/MobileLLaMA-1.4B-Chat'

# Initialize the tokenizer
tokenizer = LlamaTokenizer.from_pretrained(model_path)

# Set the device to CUDA
device = torch.device('cuda')

try:
    # Load the model with half precision for reduced memory usage and move it to the GPU
    model = LlamaForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to(device)
except RuntimeError as e:
    if 'out of memory' in str(e):
        print('OOM when loading the model. Try reducing the batch size or image resolution.')
    raise e

def generate_response(paragraph, query):
    # Concatenate the paragraph and query
    input_text = f"Given paragraph: {paragraph}\nBase on the paragraph, {query}"

    # Encode the input text
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)

    # Generate the response
    output_ids = model.generate(input_ids, max_length=200, num_return_sequences=1, do_sample=True, top_k=50, top_p=0.95, num_beams=1)[0]

    # Decode the output
    response = tokenizer.decode(output_ids, skip_special_tokens=True)

    return response

# Example usage
paragraph = "A person eating a yellow banana in a spaceship. This man looks happy and satasfying and am trying to sit on the chair, but due lack of gravity, he could not."
query = "What is the man eating?"
response = generate_response(paragraph, query)
print(response)

Given paragraph: A person eating a yellow banana in a spaceship. This man looks happy and satasfying and am trying to sit on the chair, but due lack of gravity, he could not.
Base on the paragraph, What is the man eating?

The man in the illustration is eating a yellow banana, and he is sitting on a chair in a spaceship. He is wearing a spacesuit and has a spaceship on his back. He is looking happy and sitting on the chair, which could be interpreted as a man feeling content and fulfilled in his life.
